In [13]:
import mujoco
import mediapy as media
import numpy as np

# Create the model
xml = """
<mujoco>
    <option gravity="0 0 -9.81"/>
    <asset>
        <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="512"/>
        <texture name="texplane" type="2d" builtin="checker" rgb1=".2 .3 .4" rgb2=".1 0.15 0.2" width="512" height="512" mark="cross" markrgb=".8 .8 .8"/>
        <material name="matplane" reflectance="0.3" texture="texplane" texrepeat="1 1" texuniform="true"/>
    </asset>
    <worldbody>
        <light directional="true" diffuse=".8 .8 .8" specular=".2 .2 .2" pos="0 0 5" dir="0 0 -1"/>
        <geom name="sphere" type="sphere" size="1" pos="0 0 0" rgba=".9 .9 .9 0.1"/>
        <body name="rat" pos="0 0 -1">
            <joint type="free"/>
            <geom type="ellipsoid" size=".1 .05 .025" rgba="1 0 0 1"/>
        </body>
    </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

# Create the data
data = mujoco.MjData(model)

# Simulation parameters
duration = 10  # simulation duration in seconds
timestep = model.opt.timestep
num_steps = int(duration / timestep)

# Function to generate random direction vector on internal sphere surface
def random_direction_on_sphere():
    phi = np.random.uniform(0, 2 * np.pi)
    theta = np.arccos(np.random.uniform(-1, 1))
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return np.array([x, y, z])

# Get the index of the rat body
rat_body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "rat")

# Function to project position onto internal sphere surface
def project_onto_sphere(position):
    radius = 0.9  # Slightly less than 1 to keep rat inside the sphere
    distance = np.linalg.norm(position)
    if distance > 0:
        return (radius * position) / distance
    return position

# Create a renderer
renderer = mujoco.Renderer(model)

# Set up the camera
camera = mujoco.MjvCamera()
camera.lookat = np.array([0, 0, 0])
camera.distance = 2.5
camera.elevation = -20
camera.azimuth = 45

positions = []
frames = []

# Simulation loop
for step in range(num_steps):
    # Generate random direction vector
    direction = random_direction_on_sphere()
    
    # Apply force to the rat
    force_magnitude = 2.0  # Adjusted for better movement
    force = force_magnitude * direction
    data.xfrc_applied[rat_body_id, :3] = force
    
    # Step the simulation
    mujoco.mj_step(model, data)
    
    # Get rat position and project onto sphere surface
    rat_position = data.xpos[rat_body_id]
    projected_position = project_onto_sphere(rat_position)
    
    # Update rat position to stay on sphere surface
    data.qpos[rat_body_id*3:rat_body_id*3+3] = projected_position
    
    positions.append(projected_position.copy())
    
    # # Print position (only every 100 steps to avoid cluttering the console)
    # if step % 100 == 0:
    #     print(f"Time: {data.time:.2f}s, Position: {projected_position}")
    
    # Render the scene
    renderer.update_scene(data, camera=camera)
    frame = renderer.render()
    frames.append(frame)
    
    # Clear the applied force after application
    data.xfrc_applied[rat_body_id, :3] = 0

# Show the video
media.show_video(frames, fps=60)

# Close the renderer
renderer.close()

# Verify that positions are on the surface of the sphere
for pos in positions:
    distance_from_origin = np.linalg.norm(pos)
    assert np.isclose(distance_from_origin, 0.9, atol=1e-6), f"Rat not on sphere surface: {pos}"
    print(f"Verified position on sphere: {pos}")

print("Simulation completed")

This browser does not support the video tag.


Verified position on sphere: [ 0.   0.  -0.9]
Verified position on sphere: [-9.35007682e-09  3.11579873e-06 -9.00000000e-01]
Verified position on sphere: [-7.77697239e-07  1.17647688e-05 -9.00000000e-01]
Verified position on sphere: [-2.64286429e-06  2.39884106e-05 -9.00000000e-01]
Verified position on sphere: [-4.87824202e-06  4.25970016e-05 -8.99999999e-01]
Verified position on sphere: [-4.58425400e-06  5.52777937e-05 -8.99999998e-01]
Verified position on sphere: [-4.99826159e-06  6.44948592e-05 -8.99999998e-01]
Verified position on sphere: [-8.05865083e-06  7.05045792e-05 -8.99999997e-01]
Verified position on sphere: [-1.82644318e-05  7.34010002e-05 -8.99999997e-01]
Verified position on sphere: [-3.60346923e-05  7.06263492e-05 -8.99999997e-01]
Verified position on sphere: [-6.09861374e-05  5.65069824e-05 -8.99999996e-01]
Verified position on sphere: [-9.03209770e-05  3.59564848e-05 -8.99999995e-01]
Verified position on sphere: [-1.15956262e-04  5.85500780e-06 -8.99999993e-01]
Verifi